## User TimeLine Analysing

In [1]:
import tweepy
import pandas as pd

In [3]:
app_api_key = 'dcnhAmPLv9oSecC3zQgL6itOy'
app_api_secret_key = 'zEJf0peEPhRY38jxaXSq5QI9AAiU6CgH5gRul7SLM0nyQKl1E9'


auth = tweepy.AppAuthHandler(app_api_key, app_api_secret_key)

In [5]:
def get_tweets(username: str):
    api = tweepy.API(
            auth,
            wait_on_rate_limit=True
        )

    tweets = tweepy.Cursor(
            api.user_timeline,
            screen_name = username,
            tweet_mode = 'extended',
            count = 1000
        )
    
    tweets = [tweet._json for tweet in tweets]
    df = pd.
